In [19]:
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm
from keras.utils import Sequence
from tensorflow.keras import backend
from tensorflow.keras.models import load_model
from sksurv.metrics import concordance_index_censored

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, log_loss




In [2]:
np.random.seed(1000)
tf.compat.v1.set_random_seed(1000)
tf.random.set_seed(1000)


ROOT_DIR = os.getcwd()
if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)
    
INPUT_LAYER = 'input_layer'
INPUT_SHAPE = 6


def predict_probabilities(lstm_model, sequence):
    predictions = []
    for element in list(sequence):
        element = element[np.newaxis, ...]
        predictions.append(lstm_model.predict_on_batch(x=element)[0][0])
    return np.array(predictions)


def predict_future_probabilities(lstm_model, sequence, num_steps):
    predictions = []
    sequence_length = sequence.shape[1]
    
    for element in sequence:
        element = element[np.newaxis, ..., np.newaxis]
        predictions.append(lstm_model.predict_on_batch(x=element)[0][0])
    
    for _ in range(num_steps):
        last_prediction = predictions[-1]
        input_data = np.array([[last_prediction]])
        input_data = np.repeat(input_data, sequence_length, axis=1)
        input_data = input_data[..., np.newaxis]
        output = lstm_model.predict_on_batch(input_data)
        prediction = output[0][0]
        predictions.append(prediction)
    
    return np.array(predictions)


def separate_to_positive_and_negative(data_f, patients):
    positive_patients = []
    negative_patients = []
    
    for patient in patients:
        filtered_df = data_f[data_f['patient_id'] == patient]
        unique_values = list(filtered_df['recovery_flags'].unique())
        flag = max(unique_values)
        if flag == 1.:
            positive_patients.append(patient)
        elif flag == 0.:
            negative_patients.append(patient)
        else:
            raise ValueError("Unforunately an error occured")
    return positive_patients, negative_patients

In [3]:
def predict_hazard_A1(logits):
    odd = np.exp(logits)
    hazard = odd / (1. + odd)
    return hazard

    
def predict_hazard_A0(logits):
    odd = np.exp(logits)
    hazard = odd / (1. + odd)
    return hazard


def predict_survival_A1(logits):
    hazard = predict_hazard_A1(logits)  
    surv = np.cumprod(1. - hazard)
    return surv

def predict_survival_A0(logits):
    hazard = predict_hazard_A0(logits)  
    surv = np.cumprod(1. - hazard)
    return surv


def calculate_logits(probabilities):
    probabilities = np.clip(probabilities, 1e-7, 1. - 1e-7)
    logits = np.log(probabilities / (1. - probabilities))
    return logits


In [4]:
data_1 = pd.read_csv('data_g.csv')
num_timesteps = 60

test = data_1.drop('index', axis = 1)
test_num_patients = len(test) // num_timesteps
test_patient_id = np.repeat(np.arange(test_num_patients), num_timesteps)
test['patient_id'] = test_patient_id


In [5]:
test.head()

,recovery_flags,cancer_volume,chemo_application,radio_application,sequence_lengths,chemo_dosage,radio_dosage,patient_types,patient_id
0,0.0,4.893958,0.0,0.0,59.0,0.0,0.0,2,0
1,0.0,4.966752,0.0,0.0,59.0,0.0,0.0,2,0
2,0.0,5.127451,0.0,0.0,59.0,0.0,0.0,2,0
3,0.0,5.198159,0.0,0.0,59.0,0.0,0.0,2,0
4,0.0,5.392657,0.0,0.0,59.0,0.0,0.0,2,0


In [6]:
positive_test_patients, negative_test_patients = separate_to_positive_and_negative(test, set(test_patient_id))
print("test: ", len(positive_test_patients), len(negative_test_patients))

test:  315 9685


In [7]:
negative_test_patients = negative_test_patients[:315]
reduced_test_patients = positive_test_patients + negative_test_patients

print(len(negative_test_patients))
print(len(positive_test_patients))

315
315


In [8]:
patients_probabilities = {}
best_model = load_model(os.path.join(ROOT_DIR, "best_lstm_model", "model.hdf5"))


for patient in tqdm(reduced_test_patients):
    backend.clear_session()
    
    test_sub = test[test['patient_id'] == patient]
    sequence_lengths = int(test_sub["sequence_lengths"].iloc[0])

    test_sub = test_sub.iloc[:sequence_lengths+1]
    test_sub = test_sub.drop('sequence_lengths', axis=1)
    test_sub = test_sub.drop('patient_id', axis=1)

    recovery_flag = int(test_sub["recovery_flags"].iloc[-1])
    test_sub = test_sub.drop('recovery_flags', axis=1)
    sequence = test_sub.values

    predictions = predict_probabilities(best_model, sequence)
    
    if patient not in patients_probabilities.keys():
        patients_probabilities[patient] = {"probabilities": predictions,
                                           "recovery_flag": recovery_flag}
    else:
        print("Duplicated patient: ", patient)

    best_model.reset_states()


100%|██████████| 630/630 [01:25<00:00,  7.36it/s]


In [9]:
patients_probabilities_tr = {}
best_model = load_model(os.path.join(ROOT_DIR, "best_lstm_model", "model.hdf5"))


for patient in tqdm(reduced_test_patients):
    backend.clear_session()

    test_sub = test[test['patient_id'] == patient]
    sequence_lengths = int(test_sub["sequence_lengths"].iloc[0])
    if sequence_lengths >= 1:
    
        test_sub = test_sub.iloc[:(sequence_lengths//2) + 1]
        test_sub = test_sub.drop('sequence_lengths', axis=1)
        test_sub = test_sub.drop('patient_id', axis=1)
        future_steps = sequence_lengths - (sequence_lengths//2)

        test_sub = test_sub.drop('recovery_flags', axis=1)
        sequence_tr = test_sub.values

        predictions_tr = predict_future_probabilities(best_model, sequence_tr, future_steps)
        
        if patient not in patients_probabilities_tr.keys():
            patients_probabilities_tr[patient] = {"probabilities": predictions_tr,
                                                }
        else:
            print("Duplicated patient: ", patient)

        best_model.reset_states()
    else:
        print("Insufficient sequence length for patient: ", patient)

100%|██████████| 630/630 [01:27<00:00,  7.24it/s]


In [10]:
patients_probabilities[34]

{'probabilities': array([0.02798742, 0.8482392 ], dtype=float32),
 'recovery_flag': 1}

In [11]:
def calculate_hazard_outcomes(probabilities):
    survival_probabilities = 1 - probabilities
    hazard_probabilities = np.diff(survival_probabilities)
    hazard_ratios = hazard_probabilities[1:] / hazard_probabilities[:-1]
    return survival_probabilities, hazard_probabilities, hazard_ratios

In [12]:
for patient, attr in patients_probabilities.items():
    probabilities = attr["probabilities"]
    probabilities = probabilities.flatten()
    hazard_A1 = predict_hazard_A1(probabilities)
    survival_A1 = predict_survival_A1(probabilities)  
    survival_probs, hazard_probs, hazard_ratios = calculate_hazard_outcomes(probabilities)   
    
    patients_probabilities[patient]["survival_probs"] = survival_probs
    patients_probabilities[patient]["hazard_probs"] = hazard_probs
    patients_probabilities[patient]["hazard_ratios"] = hazard_ratios

    patients_probabilities[patient]["hazard"] = hazard_A1
    
    patients_probabilities[patient]["surv"] = survival_A1

for patient, attr in patients_probabilities_tr.items():
    probabilities = attr["probabilities"]
    probabilities = probabilities.flatten()
    hazard_A1 = predict_hazard_A1(probabilities)
    survival_A1 = predict_survival_A1(probabilities)  
    survival_probs, hazard_probs, hazard_ratios = calculate_hazard_outcomes(probabilities)   
    
    patients_probabilities_tr[patient]["survival_probs"] = survival_probs
    patients_probabilities_tr[patient]["hazard_probs"] = hazard_probs
    patients_probabilities_tr[patient]["hazard_ratios"] = hazard_ratios

    patients_probabilities_tr[patient]["hazard"] = hazard_A1
    
    patients_probabilities_tr[patient]["surv"] = survival_A1


In [13]:
# Choose patient ID from 0 to 314 (315 patients in total) and run the following code:
patients_probabilities[34]

{'probabilities': array([0.02798742, 0.8482392 ], dtype=float32),
 'recovery_flag': 1,
 'survival_probs': array([0.9720126 , 0.15176082], dtype=float32),
 'hazard_probs': array([-0.82025176], dtype=float32),
 'hazard_ratios': array([], dtype=float32),
 'hazard': array([0.5069964 , 0.70019764], dtype=float32),
 'surv': array([0.4930036 , 0.14780365], dtype=float32)}

In [14]:
baseline_hazard_prob = 0.5

event_times_combined = []
model_hazard_probs_combined = []
event_observed_combined = []

for patient in tqdm(reduced_test_patients):
    test_sub = test[test['patient_id'] == patient]
    sequence_lengths = int(test_sub["sequence_lengths"].iloc[0])
    if sequence_lengths >= 2:
        test_sub = test_sub.iloc[:sequence_lengths+1]
        recovery_flag = int(test_sub["recovery_flags"].iloc[-1])
        hazard_probs = patients_probabilities[patient]["hazard"]
        event_times_combined.extend([sequence_lengths] * len(hazard_probs))
        model_hazard_probs_combined.extend(-hazard_probs)
        event_observed_combined.extend([bool(recovery_flag)] * len(hazard_probs))
    else:
        print("_")
event_times_combined = np.array(event_times_combined)
model_hazard_probs_combined = np.array(model_hazard_probs_combined)
event_observed_combined = np.array(event_observed_combined)


event_times_combined_tr = []
model_hazard_probs_combined_tr = []
event_observed_combined_tr = []

for patient in tqdm(reduced_test_patients):
    test_sub = test[test['patient_id'] == patient]
    sequence_lengths = int(test_sub["sequence_lengths"].iloc[0])
    if sequence_lengths >= 2:
        test_sub = test_sub.iloc[:sequence_lengths+1]
        recovery_flag = int(test_sub["recovery_flags"].iloc[-1])
        hazard_probs = patients_probabilities_tr[patient]["hazard"]
        event_times_combined_tr.extend([sequence_lengths] * len(hazard_probs))
        model_hazard_probs_combined_tr.extend(-hazard_probs)
        event_observed_combined_tr.extend([bool(recovery_flag)] * len(hazard_probs))
    else:
        print("_")

event_times_combined_tr = np.array(event_times_combined_tr)
model_hazard_probs_combined_tr = np.array(model_hazard_probs_combined_tr)
event_observed_combined_tr = np.array(event_observed_combined_tr)

cindex_model = concordance_index_censored(event_observed_combined, event_times_combined, model_hazard_probs_combined)
cindex_model_tr = concordance_index_censored(event_observed_combined_tr, event_times_combined_tr, model_hazard_probs_combined_tr)

print("Model C-index:", cindex_model)
print("Model C-index:", cindex_model_tr)

100%|██████████| 630/630 [00:00<00:00, 3577.35it/s]


_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_


  0%|          | 0/630 [00:00<?, ?it/s]

_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_


100%|██████████| 630/630 [00:00<00:00, 3712.68it/s]


_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
Model C-index: (0.45294077603575383, 26048944, 31461791, 275, 0)
Model C-index: (0.42559542946646217, 24476290, 33034454, 266, 0)


In [21]:
mse = mean_squared_error(true_predictions, future_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(true_predictions, future_predictions)
r2 = r2_score(true_predictions, future_predictions)
logloss = log_loss(true_labels, future_predictions)  # Only for binary outcomes

print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("R-squared:", r2)
print("Log Loss:", logloss)

NameError: name 'true_labels' is not defined

In [20]:
mse_values = []

for patient in tqdm(reduced_test_patients):
    true_predictions = patients_probabilities[patient]["hazard"]

    future_predictions = patients_probabilities_tr[patient]["hazard"]
    
    mse = mean_squared_error(true_predictions, future_predictions)
    mse_values.append(mse)

average_mse = np.mean(mse_values)
print(average_mse)

100%|██████████| 630/630 [00:00<00:00, 12846.07it/s]

0.0022517922


In [24]:
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf


libraries = [np, pd, tf, plt]
library_names = ['NumPy', 'pandas', 'TensorFlow']

versions = {}

for library, name in zip(libraries, library_names):
    version = library.__version__
    versions[name] = version

for name, version in versions.items():
    print(f"{name} version: {version}")

NumPy version: 1.24.3
pandas version: 2.0.1
TensorFlow version: 2.11.0
